In [3]:
import time
import torch
import torchvision
import torch.utils.data as Data
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn, optim

%matplotlib inline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x): # x shape: (batch, *, *, ...(
        return x.view(x.shape[0], -1)

In [11]:
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])

In [5]:
class Inception(nn.Module):
    def __init__(self, in_c, c1, c2, c3, c4):
        super(Inception, self).__init__()
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)
    
    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1,p2,p3,p4), dim=1)

In [7]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [8]:
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.Conv2d(64, 192, kernel_size=3, padding=4),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [9]:
b3 = nn.Sequential(Inception(192, 64, (96,128), (16,32), 32),
                   Inception(256, 128, (128, 192), (32,96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [10]:
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [12]:
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   GlobalAvgPool2d())

In [13]:
net = nn.Sequential(b1, b2, b3, b4, b5,
                    FlattenLayer(),
                    nn.Linear(1024,10))

In [14]:
X = torch.rand(1,1,96,96)
for blk in net.children():
    X = blk(X)
    print('output shape: ', X.shape)

output shape:  torch.Size([1, 64, 24, 24])
output shape:  torch.Size([1, 192, 15, 15])
output shape:  torch.Size([1, 480, 8, 8])
output shape:  torch.Size([1, 832, 4, 4])
output shape:  torch.Size([1, 1024, 1, 1])
output shape:  torch.Size([1, 1024])
output shape:  torch.Size([1, 10])


In [15]:
def evaluate_acc(data_iter, net, device=None):
    acc_sm, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            net.eval()
            acc_sm += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
            net.train()
            n += y.shape[0]
    return acc_sm/n

In [21]:
def train_model(net, train_iter, test_iter, batch_size, oprimizer, device, num_epochs):
    net = net.to(device)
    print('train on: ', device)
    loss_func = torch.nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sm, train_acc_sm, n, start, batch_ct = 0.0, 0.0, 0, time.time(), 0
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss_func(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sm += l.cpu().item()
            train_acc_sm += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_ct += 1
        test_acc = evaluate_acc(test_iter, net, device)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              %(epoch+1, train_l_sm/batch_ct, train_acc_sm/n, test_acc, time.time()-start))

In [22]:
batch_size = 128
rt = r'D:\notebook_canticle\Datasets\fmnist/'
def load_fmnist(batch_size, root, resize=None):
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())
    transform = torchvision.transforms.Compose(trans)
    fm_train = torchvision.datasets.FashionMNIST(root=root, train=True, transform=transform)
    fm_test  = torchvision.datasets.FashionMNIST(root=root, train=False, transform=transform)
    train_iter = Data.DataLoader(fm_train, batch_size=batch_size, shuffle=True)
    test_iter  = Data.DataLoader(fm_test,  batch_size=batch_size, shuffle=False)
    
    return train_iter, test_iter

train_iter, test_iter = load_fmnist(batch_size, rt, 96)

lr, num_epochs = 0.001, 3
optimizer = optim.Adam(net.parameters(), lr=lr)
train_model(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

train on:  cuda
epoch 1, loss 2.3027, train acc 0.105, test acc 0.100, time 89.5 sec
epoch 2, loss 1.2173, train acc 0.523, test acc 0.820, time 89.4 sec
epoch 3, loss 0.4130, train acc 0.849, test acc 0.857, time 89.3 sec
